In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# We need to install a wide variety of libraries. For this we will install pandas, numpy, seaborn and matplotlib libraries.

import numpy as np
import pandas as pd
import seaborn as sns
import string
import re
sns.set()

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings("ignore")

# Graphics in retina format are more sharp and legible
%config InlineBackend.figure_format = 'retina'

## 1- Loading the data.

In [ ]:
df = pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv", encoding="latin-1")

df = df.dropna(how="any", axis=1)
df.columns = ['target', 'message']

df.head()

## 2- Exploratory Data Analysis (EDA)

In [ ]:
df.describe()

In [ ]:
df['message_n_chars'] = df.message.apply(len) # count all chars in each sentence
df['message_n_words'] = df.message.apply(lambda sent: len(sent.split())) # count number of words in each sentence
df.head()

In [ ]:
max(df.message_n_chars), max(df.message_n_words)

In [ ]:
# Convert target to numerical variable
df['lable'] = df.target.map({'ham': 0,
                            'spam': 1})
df.head()

In [ ]:
df['target'].value_counts(normalize= True)

In [ ]:
sns.countplot(data = df, x = 'target');

**The distribution of number of chars and number of words in each sentence**

In [ ]:
sns.histplot(data= df, x= 'message_n_words', hue= 'target', multiple= 'stack');

## 3- Data Pre-processing
* Cleaning 
* removing stop words
* stemming or lemma


**Stemming/ Lematization**


For grammatical reasons, documents are going to use different forms of a word, such as write, writing and writes. Additionally, there are families of derivationally related words with similar meanings. The goal of both stemming and lemmatization is to reduce inflectional forms and sometimes derivationally related forms of a word to a common base form.

* **Stemming** usually refers to a process that chops off the ends of words in the hope of achieving goal correctly most of the time and often includes the removal of derivational affixes.

* **Lemmatization** usually refers to doing things properly with the use of a vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base and dictionary form of a word



In [ ]:
stop_words = stopwords.words('english')
stemmer    = nltk.SnowballStemmer("english")

In [ ]:
def clean_text(text):
    '''
        Make text lowercase, remove text in square brackets,remove links,remove punctuation
        and remove words containing numbers.
    '''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text) # remove urls
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # remove punctuation
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
def preprocess_data(text):
    text = clean_text(text)                                                     # Clean puntuation, urls, and so on
    text = ' '.join(word for word in text.split() if word not in stop_words)    # Remove stopwords
    text = ' '.join(stemmer.stem(word) for word in text.split())                # Stemm all the words in the sentence
    return text


In [ ]:
df['message_clean'] = df['message'].apply(preprocess_data)
df.head()


## 4- Vectorization

Currently, we have the messages as lists of tokens and now we need to convert each of those messages into a `vector` the models can work with.

First, we need to split data into train data and test data for applying modeling on it.

In [ ]:
X = df['message_clean']
y = df['lable']

X.shape, y.shape

In [ ]:
# split into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,  random_state=42)
len(X_train), len(y_train), len(X_test), len(y_test)

**CountVectorizer**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

**Tf-idf**

`TF-IDF` **Term Frequency–Inverse Document Frequency**, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.
The tf–idf value increases proportionally to the number of times a word appears in the document and is offset by the number of documents in the corpus that contain the word, which helps to adjust for the fact that some words appear more frequently in general.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
X_train_tfidf = tfidf.fit_transform(X_train_cv)

## 5- Modeling

### multinomial Naive Bayes
We will use `multinomial Naive Bayes`


The multinomial Naive Bayes classifier is suitable for classification with discrete features

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_cv, y_train)

In [ ]:
# predict classes
y_pred = nb.predict(X_test_cv)
y_pred_proba = nb.predict_proba(X_test_cv)[:, 1]

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

In [ ]:
p, r, f, _ = precision_recall_fscore_support(y_test, y_pred,
                                                 average='macro')
print(f'precision is  -> {round(p,2)}')
print(f'recall is -> {round(r,2)}')
print(f'f1_score is -> {round(f,2)}')

In [ ]:
# calculate the accuracy 
acc = accuracy_score(y_test, y_pred)
print(f'Accuracy is -> {round(acc,2)}')

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print('The confusion_matrix is: \n', cm);

In [ ]:
from mlxtend.plotting import plot_confusion_matrix

fig, ax = plot_confusion_matrix(conf_mat= cm, colorbar=True)
plt.show()

In [ ]:
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f'The roc_auc Score is -> {round(roc_auc,2)}')

### Make it by pipline

**`MultinomialNB with TF-idf`**

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

pipe = Pipeline([('bow', CountVectorizer()), 
                 ('tfidf', TfidfTransformer()),  
                 ('model', MultinomialNB())])

pipe.fit(X_train, y_train)

In [ ]:
y_pred = pipe.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
confusion_matrix(y_test, y_pred)

### XGboost

**`XGboost with TF-idf`**

In [ ]:
import xgboost as xgb

pipe = Pipeline([
                ('bow', CountVectorizer()), 
                ('tfidf', TfidfTransformer()),  
                ('model', xgb.XGBClassifier(
                    learning_rate=0.1,
                    max_depth=7,
                    n_estimators=80,
                    use_label_encoder=False,
                    eval_metric='auc'))
                ])


In [ ]:
# Fit the pipeline with the data
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
# y_pred_train = pipe.predict(X_train)

# print('Train: {}'.format(accuracy_score(y_train, y_pred_train)))
print(f'Accuracy Score For Test Data: -> {round(accuracy_score(y_test, y_pred),2)}')
print(f'Confusion Matrix Score For Test Data:\n {confusion_matrix(y_test, y_pred)}')

In [ ]:
# calculate the accuracy 
acc = accuracy_score(y_test, y_pred)
print(f'Accuracy is -> {round(acc,2)}')

The **multinomial Naive Bayes** and the **XGboost** have the same accuracy score *`0.97`*

## BERT "Bidirectional Encoder Representations from Transformers"

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

import transformers
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

I can also using `AutoTokenizer`

the `AutoTokenizer` class **will grab the proper tokenizer class in the library based on the checkpoint** name, and can be used directly with any checkpoint:



In [ ]:
def bert_encode(input_text, max_len):
    input_ids = []
    attension_masks = []
    for text in input_text:
        output_dict = tokenizer.encode_plus(
            text, 
            add_special_tokens = True,
            truncation=True,
            max_length = max_len,
            pad_to_max_length = True,
            return_attention_mask = True
        )
        input_ids.append(output_dict['input_ids'])
        attension_masks.append(output_dict['attention_mask'])
    return np.array(input_ids), np.array(attension_masks)

In [ ]:
text = df['message_clean']
target = df['lable']
train_input_ids, train_attention_masks = bert_encode(text, 60)


In [ ]:
def create_model(bert_model):
    input_ids = tf.keras.Input(shape= (60,), dtype= 'int32')
    attention_masks = tf.keras.Input(shape= (60,), dtype= 'int32')
    
    output = bert_model([input_ids, attention_masks])
    output = output[1]
    output = tf.keras.layers.Dense(32, activation= 'relu')(output)
    output = tf.keras.layers.Dropout(0.2)(output)
    output = tf.keras.layers.Dense(1, activation= 'sigmoid')(output)
    
    model = tf.keras.models.Model(inputs= [input_ids, attention_masks], outputs= output)
    model.compile(Adam(lr=1e-5), loss= 'binary_crossentropy', metrics= ['accuracy'])
    return model

In [ ]:
from transformers import TFBertModel
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

In [ ]:
model = create_model(bert_model)
model.summary()

In [ ]:
%%time

history = model.fit(
    [train_input_ids, train_attention_masks],
    target, 
    validation_split = 0.2,
    epochs = 3,
    batch_size = 10
)

### BOOOM!!
The accuracy score of *BERT* reach to **`0.99`**